In [1]:
import torch
import h5py
import numpy as np
import os
from torch import nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
from visdom import Visdom
import time

In [2]:
files = os.listdir("/root/autodl-tmp/Datasets")
files.sort()

In [3]:
len(files)

4302

In [4]:
MAX = 4000
i = 0

In [5]:
DATA_train = []
DATA_test  = []

In [6]:
for file in files:
    filename = os.path.splitext(file)
    if filename[1] == '.hdf5':
        path = os.path.join('/root/autodl-tmp/Datasets/' + file)
        f = h5py.File(path)
        data = f['dataset']
        if i < MAX:
            DATA_train.append(data)
        else:
            DATA_test.append(data)
        i += 1 

In [7]:
len(DATA_train), len(DATA_test)

(4000, 302)

In [8]:
type(DATA_train)

list

In [9]:
DATA_train[0][0]

array([ 1.0000000e+00,  2.9456370e+21,  7.9943233e+00,  1.4509925e+07,
       -1.9568456e+11,  2.4651726e+07, -1.6436408e+08, -4.3989524e+07,
       -1.0778338e+07,  2.7425510e+02,  4.4113613e+04,  1.7417133e+02],
      dtype=float32)

In [10]:
def convertToArray(data):
    assert type(data) != np.ndarray
    return np.array(data)

In [11]:
def randomGetData(data, size=100):
    assert size <= 100
    index = np.random.randint(0, len(data), size)
    return data[index]

In [12]:
# def convertToArray(data):
#     DATA = np.array(data[0])
#     for i in range(1, len(data)):
#         data[i] = np.array(data[i])
#         DATA = np.concatenate((DATA, data[i]))
#     return DATA

In [12]:
DATA_train_PRE_01 = np.array(DATA_train[:100])

In [13]:
DATA_test_PRE_01 = np.array(DATA_test[:50])

In [ ]:
# DATA_train_PRE_02 = np.array(DATA_train[100:200])

In [14]:
DATA_train_PRE_01.shape

(100, 242311, 12)

In [15]:
DATA_test_PRE_01.shape

(50, 242311, 12)

In [75]:
# norm function
def data_norm(data):
    mean = data.mean(dim=1, keepdim=True)
    std = data.std(dim=1, keepdim=True)
    return (data-mean)/(std+1)

In [16]:
# 将坐标作为输入 势能作为输出
x_train_data_pos = DATA_train_PRE_01[:,:,-6:-3]
y_train_data_pot = DATA_train_PRE_01[:,:,4]

In [17]:
x_train_data_pos.shape, y_train_data_pot.shape

((100, 242311, 3), (100, 242311))

In [18]:
x_train_data_pos = torch.from_numpy(x_train_data_pos)
y_train_data_pot = torch.from_numpy(y_train_data_pot)

In [19]:
x_train_data_pos.shape, y_train_data_pot.shape

(torch.Size([100, 242311, 3]), torch.Size([100, 242311]))

In [20]:
y_train_data_pot = y_train_data_pot.reshape(100, 242311, 1)
y_train_data_pot.shape

torch.Size([100, 242311, 1])

In [21]:
x_test_data_pos = DATA_test_PRE_01[:,:,-6:-3]
y_test_data_pot = DATA_test_PRE_01[:,:,4].reshape(50, 242311, 1)

In [22]:
x_test_data_pos = torch.from_numpy(x_test_data_pos)
y_test_data_pot = torch.from_numpy(y_test_data_pot)

In [23]:
x_test_data_pos.shape, y_test_data_pot.shape

(torch.Size([50, 242311, 3]), torch.Size([50, 242311, 1]))

In [84]:
x_train_data_pos = data_norm(x_train_data_pos)
y_train_data_pot = data_norm(y_train_data_pot)
x_test_data_pos = data_norm(x_test_data_pos)
y_test_data_pot = data_norm(y_test_data_pot)

In [24]:
x_train_data_pos.max(), x_train_data_pos.min()

(tensor(4.1418e+09), tensor(-3.5442e+09))

In [25]:
y_train_data_pot.max(), y_train_data_pot.min()

(tensor(-9.5733e+09), tensor(-2.3961e+11))

In [26]:
x_test_data_pos.max(), x_test_data_pos.min()

(tensor(2.9852e+10), tensor(-2.5352e+10))

In [27]:
y_test_data_pot.max(), y_test_data_pot.min()

(tensor(-1.3034e+09), tensor(-2.3782e+11))

In [28]:
y_train_data_pot.shape

torch.Size([100, 242311, 1])

In [29]:
class MyDataset(Dataset):
    def __init__(self, x, label):
        self.x = x
        self.label = label
    def __len__(self):
        return len(self.x)
    def __getitem__(self, item):
        return self.x[item], self.label[item]

In [30]:
train_data_01 = MyDataset(x_train_data_pos, y_train_data_pot)

In [31]:
type(train_data_01)

__main__.MyDataset

In [32]:
test_data_01 = MyDataset(x_test_data_pos, y_test_data_pot)

In [33]:
# Dataloader
train_dataloader = DataLoader(train_data_01, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_data_01, batch_size=1, shuffle=False)

In [34]:
len(train_dataloader)

100

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [36]:
# network model
class MyNetwork(nn.Module):
    def __init__(self, input_units=3, output_units=1):
        super(MyNetwork, self).__init__()
        self.fc_unit = nn.Sequential(nn.Linear(input_units, 120), 
                                     nn.Tanh(), 
                                     nn.Linear(120, 100), 
                                     nn.Tanh(), 
                                     nn.Linear(100, 50), 
                                     nn.Tanh(), 
                                     nn.Linear(50, output_units)
                                    )
    def forward(self, x):
        x = self.fc_unit(x)
        return x
         

In [37]:
model = MyNetwork().to(device)
model

MyNetwork(
  (fc_unit): Sequential(
    (0): Linear(in_features=3, out_features=120, bias=True)
    (1): Tanh()
    (2): Linear(in_features=120, out_features=100, bias=True)
    (3): Tanh()
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): Tanh()
    (6): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [38]:
model(x_train_data_pos[0].to(device)).shape

torch.Size([242311, 1])

In [39]:
loss_fn = nn.MSELoss().to(device)

In [40]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [41]:
viz = Visdom()
viz.line([0.], [0.], win='train_loss', opts=dict(title='train_loss'))
viz.line([[0.0, 1.0]], [0.], win='test', opts=dict(title='test_loss&acc.', 
                                                   legend=['loss', 'acc.']))

Setting up a new session...
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/connection.

Exception in user code:
------------------------------------------------------------


[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
Visdom python client failed to establish socket to get messages from the server. This feature is optional and can be disabled by initializing Visdom with `use_incoming_socket=False`, which will prevent waiting for this request to timeout.


Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/root/miniconda3/lib/python3.8/site-packages/urllib3/connection.py", line 234, in request
  

False

[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99] Cannot assign requested address
on_close() takes 1 positional argument but 3 were given
[Errno 99]

In [42]:
global_step = 0. # 记录训练步长

In [43]:
train_data = train_dataloader
test_data = test_dataloader

In [44]:
len(train_data)

100

In [46]:
epochs = 1000

In [ ]:
for epoch in range(epochs):
    model.train()  # train
    for batch_idx, (x, label) in enumerate(train_dataloader):
        x, label = x.to(device), label.to(device)
        y_hat = model(x)
        # label = torch.flatten(label, 1, -1)
        train_loss = loss_fn(y_hat, label)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        global_step += 1
        viz.line([train_loss.item()], [global_step], win='train_loss', update='append')
    if batch_idx % 100 == 0:    
        print(epoch, 'loss: {:.2f}'.format(loss.item()))
    model.eval()   # test
    with torch.no_grad():
        correct = 0
        test_loss = 0
        total_correct = 0
        total_num = 0
        for x, label in test_dataloader:
            x, label = x.to(device), label.to(device)
            pred = model(x)
            # label = torch.flatten(label, 1, -1)
            loss = loss_fn(pred, label)
            test_loss += loss_fn(pred, label).item()
            if loss < 0.5:
                correct += 1
            # correct = torch.eq(pred,label).float().sum().item()
            total_correct += correct
            total_num += x.size(0)   
        test_loss /= total_num
        acc = total_correct / total_num
        # visdom 可视化
        viz.line([[test_loss, acc]], [global_step], win='test', update='append')
        if batch_idx % 100 == 0:
            print(epoch, 'test acc: {:.2f}%'.format(acc*100))
            print('test loss: {:.2f}'.format(loss))